In [1]:
import math
import numpy as np

In [2]:
NodesCondition = []
ForcesCondition = []
def UBoundaryCondition(nU,u,i):
    nU[i][0] = u
    NodesCondition.append(i)

def FBoundaryCondition(nF,f,i):
    nF[i][0] = f
    ForcesCondition.append(i)
    
def AssemblyStiffness(nStiffnessMatrix,k,i):
    p1 = index['K'+str(i)][0]
    p2 = index['K'+str(i)][1]
    p3 = index['K'+str(i)][2]
    nindex = [p1,p2,p3]
    for j in range(p1,p2+1):
        for m in range(p1,p2+1):
            nStiffnessMatrix[nindex[j-p1]][nindex[m-p1]] += k[j-p1][m-p1]
            #nStiffnessMatrix[j][p] += k[j-p1][p-p1]
    
def Initialize(nStiffnessMatrix,nU,nF):
    for i in range(0,Nodes):
        nU[i][0] = math.pi
        nF[i][0] = math.pi
    for i in range(0,NumberOfElement):
        AssemblyStiffness(nStiffnessMatrix,K[i],i)

def PreSolvingStiffness(nStiffnessMatrix):
    nsize = Nodes-len(NodesCondition)
    newStiffness = np.zeros((nsize,nsize))
    contr = -1
    for i in range(0,Nodes):
        contc = -1
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        for j in range(0,Nodes):
            flagc = False
            for k in range(0,len(NodesCondition)):
                if(j == NodesCondition[k]):
                    flagc = True
                    break
            if(flagc):
                continue
            contc += 1
            newStiffness[contr][contc] = nStiffnessMatrix[i][j]
    return newStiffness

def PreSolvingF(nF,nS,nU):
    nsize = Nodes-len(NodesCondition)
    newF = np.zeros(nsize).reshape(nsize,1)
    contr = -1
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            for k in range(0,Nodes):
                nF[k][0] = nF[k][0]-nS[k][i]*nU[i][0]
            continue

    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        newF[contr][0] = nF[i][0]
    
    return newF
                      

def Solve(nStiffnessMatrix,nU,nF):
    newStiffness = PreSolvingStiffness(nStiffnessMatrix)
    newF = PreSolvingF(nF,nStiffnessMatrix,nU)
    u = np.linalg.solve(newStiffness,newF)
    contr = -1
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        nU[i][0] = u[contr][0]
    nnF = np.matmul(StiffnessMatrix,nU)
    return nU,nnF

In [3]:
NodesCondition = []
Nodes = 5
NumberOfElement = (Nodes-1)//2

E = 210*1e6 #GPA
A = 0.003 #m2
L = 4 #m
dL = np.ones(NumberOfElement)*L/NumberOfElement

K = []
index = {}
for i in range(NumberOfElement):
    K.append(E*A/(3*dL[i])*np.array([
        [7,1,-8],
        [1,7,-8],
        [-8,-8,16]
    ]))
    index["K"+str(len(K)-1)] = [2*i,2*i+2,2*i+1]

StiffnessMatrix = np.zeros((Nodes,Nodes))
U = np.zeros(Nodes).reshape(Nodes,1)
F = np.zeros(Nodes).reshape(Nodes,1)

Initialize(StiffnessMatrix,U,F)
UBoundaryCondition(U,0,0)
FBoundaryCondition(F,5,1)
FBoundaryCondition(F,-10,2)
FBoundaryCondition(F,-7,3)
FBoundaryCondition(F,10,4)

U,F=Solve(StiffnessMatrix,U,F)

print("Stiffness Matrix:\n",StiffnessMatrix,'\n')
print("Displacements:\n",U,'\n')
print("Forces:\n",F)

Stiffness Matrix:
 [[ 735000. -840000.  105000.       0.       0.]
 [-840000. 1680000. -840000.       0.       0.]
 [ 105000. -840000. 1470000. -840000.  105000.]
 [      0.       0. -840000. 1680000. -840000.]
 [      0.       0.  105000. -840000.  735000.]] 

Displacements:
 [[ 0.00000000e+00]
 [-4.16666667e-06]
 [-1.42857143e-05]
 [-8.13492063e-06]
 [ 6.34920635e-06]] 

Forces:
 [[  2.]
 [  5.]
 [-10.]
 [ -7.]
 [ 10.]]


In [4]:
NodesCondition = []
Nodes = 5
NumberOfElement = (Nodes-1)//2

E = 210*1e6 #GPA
A = 0.003 #m2
#L = 4 #m
#dL = np.ones(NumberOfElement)*L/NumberOfElement
dL = np.array([1.5,1])

K = []
index = {}
for i in range(NumberOfElement):
    K.append(E*A/(3*dL[i])*np.array([
        [7,1,-8],
        [1,7,-8],
        [-8,-8,16]
    ]))
    index["K"+str(len(K)-1)] = [2*i,2*i+2,2*i+1]

StiffnessMatrix = np.zeros((Nodes,Nodes))
U = np.zeros(Nodes).reshape(Nodes,1)
F = np.zeros(Nodes).reshape(Nodes,1)

Initialize(StiffnessMatrix,U,F)
UBoundaryCondition(U,0,0)
UBoundaryCondition(U,0.002,4)
FBoundaryCondition(F,-10,2)
FBoundaryCondition(F,0,1)
FBoundaryCondition(F,0,3)

U,F=Solve(StiffnessMatrix,U,F)

print("Stiffness Matrix:\n",StiffnessMatrix,'\n')
print("Displacements:\n",U,'\n')
print("Forces:\n",F)

Stiffness Matrix:
 [[  980000. -1120000.   140000.        0.        0.]
 [-1120000.  2240000. -1120000.        0.        0.]
 [  140000. -1120000.  2450000. -1680000.   210000.]
 [       0.        0. -1680000.  3360000. -1680000.]
 [       0.        0.   210000. -1680000.  1470000.]] 

Displacements:
 [[0.        ]
 [0.00059524]
 [0.00119048]
 [0.00159524]
 [0.002     ]] 

Forces:
 [[-5.00000000e+02]
 [ 0.00000000e+00]
 [-1.00000000e+01]
 [-4.54747351e-13]
 [ 5.10000000e+02]]
